In [0]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import zipfile, os

# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

zip_id = '1UlSfj78z0sBV3ywrOMqXEi57RyGxhVKd'
print ("Downloading zip file")
myzip = drive.CreateFile({'id': zip_id})
myzip.GetContentFile('model.zip')
print ("Uncompressing zip file")
zip_ref = zipfile.ZipFile('model.zip', 'r')
zip_ref.extractall()
zip_ref.close()

Uncompressing zip file


In [0]:
np_load_old = np.load


np.load = lambda *a,**k: np_load_old(*a, allow_pickle=True, **k)

X=np.load("X.npy")

word2id=np.load("word2id.npy")
word2id=dict((word2id[None])[0])
np.load = np_load_old

id2word = {v: k for k, v in word2id.items()}

In [0]:
import numpy as np
from pylab import random
import time
N=X.shape[0]
M=X.shape[1]
K = 10    
maxIteration = 30
threshold = 10.0
topicWordsNum = 10

In [0]:
lamda = random([N, K])

for i in range(N):
    lamda[i, :] /= sum(lamda[i, :]);
   

theta = random([K, M])

for i in range(K):
    theta[i, :] /= sum(theta[i, :])


p = np.zeros([N, M, K])

In [0]:
def EStep():
    for i in range(0, N):
        for j in range(0, M):
            denominator = 0;
            for k in range(0, K):
                p[i, j, k] = theta[k, j] * lamda[i, k];
                denominator += p[i, j, k];
            if denominator == 0:
                for k in range(0, K):
                    p[i, j, k] = 0;
            else:
                for k in range(0, K):
                    p[i, j, k] /= denominator;

In [0]:
def MStep():
   
    for k in range(0, K):
        denominator = 0
        for j in range(0, M):
            theta[k, j] = 0
            for i in range(0, N):
                theta[k, j] += X[i, j] * p[i, j, k]
            denominator += theta[k, j]
        if denominator == 0:
            for j in range(0, M):
                theta[k, j] = 1.0 / M
        else:
            for j in range(0, M):
                theta[k, j] /= denominator
        
    
    for i in range(0, N):
        for k in range(0, K):
            lamda[i, k] = 0
            denominator = 0
            for j in range(0, M):
                lamda[i, k] += X[i, j] * p[i, j, k]
                denominator += X[i, j];
            if denominator == 0:
                lamda[i, k] = 1.0 / K
            else:
                lamda[i, k] /= denominator


In [0]:
def LogLikelihood():
    loglikelihood = 0
    for i in range(0, N):
        for j in range(0, M):
            tmp = 0
            for k in range(0, K):
                tmp += theta[k, j] * lamda[i, k]
            if tmp > 0:
                loglikelihood += X[i, j] * np.log(tmp)
    return loglikelihood

In [0]:
oldLoglikelihood = 1
newLoglikelihood = 1
for i in range(0, maxIteration):
    EStep()
    MStep()
    newLoglikelihood = LogLikelihood()
    print("[", time.strftime('%Y-%m-%d %H:%M:%S',time.localtime(time.time())), "] ", i+1, " iteration  ", str(newLoglikelihood))
    if(oldLoglikelihood != 1 and newLoglikelihood - oldLoglikelihood < threshold):
        break
    oldLoglikelihood = newLoglikelihood

[ 2019-12-05 11:33:52 ]  1  iteration   -147587.9804790245
[ 2019-12-05 11:34:54 ]  2  iteration   -145377.72377402996
[ 2019-12-05 11:35:57 ]  3  iteration   -142266.21796475846
[ 2019-12-05 11:37:01 ]  4  iteration   -138433.2937099662
[ 2019-12-05 11:38:04 ]  5  iteration   -134624.2768299272
[ 2019-12-05 11:39:07 ]  6  iteration   -131531.9600974335
[ 2019-12-05 11:40:10 ]  7  iteration   -129305.70218162914
[ 2019-12-05 11:41:13 ]  8  iteration   -127753.85208206631
[ 2019-12-05 11:42:16 ]  9  iteration   -126654.60695501043
[ 2019-12-05 11:43:19 ]  10  iteration   -125854.54053576935
[ 2019-12-05 11:44:22 ]  11  iteration   -125247.73157169727
[ 2019-12-05 11:45:25 ]  12  iteration   -124777.44897796257
[ 2019-12-05 11:46:29 ]  13  iteration   -124404.17846557255
[ 2019-12-05 11:47:32 ]  14  iteration   -124115.70193074385
[ 2019-12-05 11:48:35 ]  15  iteration   -123907.01620853985
[ 2019-12-05 11:49:38 ]  16  iteration   -123755.50064833486
[ 2019-12-05 11:50:41 ]  17  iteratio

In [0]:
p_d=1/100
tmp=np.zeros((N,K,M))
for j in range(M):
 for k in range(K):
  for i in range (N):
    tmp[i,k,j]=p_d*lamda[i,k]*theta[k,j]

tmp2=np.zeros((K,M))
for k in range(K):
  for j in range(M):
    tmp2[k,j]=np.sum(tmp[:,k,j])

proba=np.zeros((K,M))
for k in range(K):
  for j in range(M):
    proba[k,j]=tmp2[k,j]/np.sum(tmp2[:,j]) 

import matplotlib.pyplot as pp

ar=proba.reshape(-1)
pp.plot(ar, len(ar) * [1], "x")
pp.show()      


ar=lamda.reshape(-1)
pp.plot(ar, len(ar) * [1], "x")
pp.show()

In [0]:
import matplotlib.pyplot as pp

ar=proba.reshape(-1)[0:int(len(proba.reshape(-1))  *(0.05) )]
pp.plot(ar, len(ar) * [1], "x")
pp.show()

In [0]:
ar=lamda.reshape(-1)
pp.plot(ar, len(ar) * [1], "x")
pp.show()